## Simulation as Optimization: Finding Paths of Least Action with Gradient Descent
Tim Strang and Sam Greydanus | 2023 | MIT License

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch, time

from celluloid import Camera
from IPython.display import HTML
from base64 import b64encode

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from main import * # SimOpt code

In [4]:
def plot_action_stats(ax, S, T, V, S_ode, T_ode, V_ode):
    alpha = .7
    plot_config = [(S, 'k', '$S$', alpha), (T, 'm', '$\sum_i T_i$', alpha),
                   (-np.asarray(V), 'c', '$-\sum_i V_i$', alpha),
                   (S_ode, 'k--', '$S$ (ODE)', 1), (T_ode, 'm--', '$-\sum_i T_i$ (ODE)', 1),
                   (-V_ode, 'c--', '$-\sum_i V_i$ (ODE)', 1)]
    N = len(S)
    for i, (x, fmt, label, alpha) in enumerate(plot_config):
        if i <= 2:
            ax.plot(np.arange(N), x, fmt, alpha=alpha, linewidth=4, label=label)
        else:
            ax.plot([0,N], [x]*2, fmt, alpha=alpha, linewidth=4, label=label)
            
def plot_helper(ax, ax_labels, fontsz, legend=False):
    plt.title(ax_labels['title'], fontweight="bold")
    if legend:
        plt.legend(ncol=2, fontsize=fontsz['legend'])
    plt.xlabel(ax_labels['x_label'])
    plt.ylabel(ax_labels['y_label'])
    ax.xaxis.label.set_fontsize(fontsz['x_label'])
    ax.yaxis.label.set_fontsize(fontsz['y_label'])
    ax.title.set_fontsize(fontsz['title'])
    ax.tick_params(axis='both', length=9, width=3, labelsize=15)
    plt.tight_layout()  # helps clean up plots sometimes
    
def action_plot(info, x_sim, ax_labels, fontsz, L_fn, legend=False, fig=None):
    if fig is None:
        fig = plt.figure(figsize=(8.3333, 6.25), dpi=50)
    ax = fig.add_subplot(111)
    S_ode, T_ode, V_ode = action(torch.tensor(x_sim), L_fn=L_fn, dt=dt)
    plot_action_stats(ax, info['S'], info['T'], info['V'], S_ode.sum(), T_ode.sum(), V_ode.sum())

    plot_helper(ax, ax_labels, fontsz, legend=legend)
    return fig

## Free body

In [5]:
dt = 0.25 ; N = 1 ; steps = 60
t_sim, x_sim = simulate_freebody(dt=dt, steps=steps)
init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1.5e0, zero_basepath=True) # [time, N*2]
t_min, path, xs_min, info = minimize_action(init_path, steps=500, step_size=1e0, 
                                       L_fn=lagrangian_freebody, dt=dt, opt='adam', verbose=False)

TypeError: minimize_action() got an unexpected keyword argument 'verbose'

In [ ]:
fig = plt.figure(figsize=(8.3333, 6.25), dpi=300)
M_sim = int(len(x_sim)/4)

ax1 = fig.add_subplot(241)
ax2 = fig.add_subplot(242)
ax3 = fig.add_subplot(243)
ax4 = fig.add_subplot(244)
ax5 = fig.add_subplot(245)
ax6 = fig.add_subplot(246)
ax7 = fig.add_subplot(247)
ax8 = fig.add_subplot(248)

sim_axes=[ax1, ax2, ax3, ax4]
min_axes=[ax5, ax6, ax7, ax8]

for i, ax in enumerate(sim_axes):
    j = i+1
    ax.plot(t_sim[:j*M_sim], x_sim[:j*M_sim], 'k.-')
    ax.set_xlim(-1, t_sim.max()+1)
    ax.set_ylim(-5, 40)

ax5.plot(t_min, xs_min[0], 'k.-')
ax6.plot(t_min, xs_min[3], 'k.-')
ax7.plot(t_min, xs_min[6], 'k.-')
ax8.plot(t_min, xs_min[-1], 'k.-')


plt.show()


In [ ]:
fig = plt.figure(figsize=(8.3333, 6.25), dpi=300) 
ax = fig.add_subplot(111)
name = 'Free body'
ax.plot(t_sim, x_sim, 'r-', label='ODE solution', linewidth=3)
ax.plot(t_min, xs_min[0], 'y.-', alpha=.3, label='Initial (random) path')
for i, xi in enumerate(xs_min):
    label = 'During optimization' if i==10 else None
    ax.plot(t_min, xi, alpha=.3 + .7 * i/(len(xs_min)-1), color=plt.cm.viridis( 1-i/(len(xs_min)-1) ), label=label)
ax.plot(t_min, xs_min[-1], 'b.-', label='Final (optimized) path')
ax.plot(t_min[[0,-1]], xs_min[0].data[[0,-1]], 'b+', markersize=15, label='Points held constant')

plt.ylim(-5, 40)

fontsz = {'title': 23, 'x_label': 23, 'y_label': 23, 'legend': 10}
ax_labels = {'title':'Free Body Height vs Time',
             'x_label':'Time (s)', 'y_label':'Height (m)'}
plot_helper(ax, ax_labels, fontsz, legend=True)
path ='./dynamic/{}.pdf'.format(name.lower().replace(' ', ''))
plt.show() ; fig.savefig(path)

In [ ]:
ax_labels = {'title':'Action and associated quantities ({})'.format(name),
                 'x_label':'Optimizer Steps', 'y_label':'J * s'}
fontsz = {'title': 17, 'x_label': 23, 'y_label': 28, 'legend': 14}
fig = action_plot(info, x_sim, ax_labels, fontsz, L_fn=lagrangian_freebody, legend=True)
plt.ylim(-25, 20)
path ='./static/{}_action.pdf'.format(name.lower().replace(' ', ''))
plt.show() ; fig.savefig(path)

## Single pendulum

In [ ]:
dt = 1 ; N = 1
t_sim, x_sim = simulate_pend(dt=dt)

init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1.5e0, zero_basepath=False, clip_rng=.5) # [time, N*2]
t_min, path, xs_min, info = minimize_action(init_path, steps=100, step_size=1e0, 
                                            L_fn=lagrangian_pend, dt=dt, opt='adam', 
                                            loss_coeffs=(1,1), verbose=False)

fig = plt.figure(figsize=(8.3333, 6.25), dpi=300) 
ax = fig.add_subplot(111)
name = 'Pendulum'
ax.plot(t_sim, np.sin(x_sim), 'r-', label='ODE solution', linewidth=5)
ax.plot(t_min, np.sin(xs_min[0]), 'y.-', alpha=.3, label='Initial (random) path')
for i, xi in enumerate(xs_min):
    label = 'During optimization' if i==10 else None
    ax.plot(t_min, np.sin(xi), alpha=.3 + .7 * i/(len(xs_min)-1), color=plt.cm.viridis( 1-i/(len(xs_min)-1) ), label=label)
ax.plot(t_min, np.sin(xs_min[-1]), 'b.-', label='Final (optimized) path')
ax.plot(t_min[[0,-1]], np.sin(xs_min[0].data[[0,-1]]), 'b+', markersize=15, label='Points held constant')

fontsz = {'title': 19, 'x_label': 23, 'y_label': 23, 'legend': 10}
ax_labels = {'title':'Pendulum Height vs Time',
             'x_label':'Time (s)', 'y_label':'Height (m)'}
plot_helper(ax, ax_labels, fontsz)
path ='./dynamic/{}.pdf'.format(name.lower().replace(' ', ''))
plt.show() ; fig.savefig(path)

In [ ]:
ax_labels = {'title':'Action and associated quantities ({})'.format(name),
                 'x_label':'Optimizer Steps', 'y_label':'J * s'}
fontsz = {'title': 18, 'x_label': 23, 'y_label': 28, 'legend': 10}
fig = action_plot(info, x_sim, ax_labels, fontsz, L_fn=lagrangian_pend)
plt.ylim(-200, 600)
path ='./static/{}_action.pdf'.format(name.lower().replace(' ', ''))
plt.show() ; fig.savefig(path)

## Double pendulum

In [ ]:
#make_video(radial2cartesian(x_sim), path='sim.mp4', interval=60, ms=20)
#mp4 = open('sim.mp4','rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
dt = 0.06 ; N = 2
t_sim, x_sim = simulate_dblpend(dt=dt)

init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1e0, zero_basepath=False, unpert_till=3) # [time, N*2]
t_min, path, xs_min, info = minimize_action(init_path, steps=200, step_size=1e-1, 
                                            L_fn=lagrangian_dblpend, dt=dt, opt='adam',
                                            loss_coeffs=(1,0), verbose=False)

fig = plt.figure(figsize=(8.3333, 6.25), dpi=300)
ax = fig.add_subplot(111)
name = 'Double pendulum'
nm, xy = (1, 1)
radsim = radial2cartesian(x_sim)[:, nm, xy]
size = len(radsim)
ax.plot(t_sim, radsim, 'r-', label='ODE solution', linewidth=5)
ax.plot(t_min, radial2cartesian(xs_min[0])[:, nm, xy], 'y.-', alpha=.3, label='Initial (random) path')
for i, xi in enumerate(xs_min):
    label = 'During optimization' if i==10 else None
    ax.plot(t_min, radial2cartesian(xs_min[i])[:, nm, xy],
             alpha=.3 + .7 * i/(size - 1), 
             color=plt.cm.viridis( 1-i/(size - 1)), label=label)
ax.plot(t_min, radial2cartesian(xs_min[-1])[:, nm, xy], 'b.-', label='Final (optimized) path')
ax.plot(t_min[[0,-1]], radial2cartesian(xs_min[0])[[0,-1], nm, xy],
         'b+', markersize=15, label='Points held constant')

ax_labels = {'title':'Double Pendulum Height vs Time',
             'x_label':'Time (s)', 'y_label':'Height (m)'}
fontsz = {'title': 18, 'x_label': 23, 'y_label': 23, 'legend': 10}
plot_helper(ax, ax_labels, fontsz)

path ='./dynamic/{}.pdf'.format(name.lower().replace(' ', ''))
plt.show() ; fig.savefig(path)

#make_video(radial2cartesian(xs_min[-1]), path='sim.mp4', interval=60, ms=20)
#mp4 = open('sim.mp4','rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
name = 'Double pendulum'
ax_labels = {'title':'Action and associated quantities ({})'.format(name),
                 'x_label':'Optimizer Steps', 'y_label':'J * s'}
fontsz = {'title': 16, 'x_label': 23, 'y_label': 28, 'legend': 10}
fig = action_plot(info, x_sim, ax_labels, fontsz, L_fn=lagrangian_dblpend)
plt.ylim(0, 7)
path ='./static/{}_action.pdf'.format(name.lower().replace(' ', '')) ; print(path)
plt.show() ; fig.savefig(path)

## Three body problem

In [ ]:
t, x = simulate_3body()
#make_video(x, path='sim.mp4', interval=60, ms=20)
#mp4 = open('sim.mp4','rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
dt = 0.5 ; N = 3

t_sim, x_sim = simulate_3body(dt=dt)
init_path = PerturbedPath(x_sim, N=N, sigma=2e-2) # [time, N*2]
t_min, path, xs_min, info = minimize_action(init_path, steps=125, step_size=1e1,
                                       L_fn=lagrangian_3body, dt=dt, opt='sgd')

In [ ]:

fig = plt.figure(figsize=(8.3333, 6.25), dpi=300)
ax = fig.add_subplot(111)

N = x_sim.shape[-2]

k = 30
ax.plot(t_min, x_sim.reshape(-1,N*2)[...,k], 'r-', label='ODE solution', linewidth=5)
ax.plot(t_min, xs_min[0].reshape(-1,N*2)[...,k], 'y.-', alpha=.3, label='Initial (random) path')
for i, xi in enumerate(xs_min):
    label = 'During optimization' if i==10 else None
    ax.plot(t_min, xs_min[i].detach().numpy().reshape(-1,N*2)[...,k], alpha=.3 + .7 * i/(17), color=plt.cm.viridis( 1-i/(17) ), label=label)
ax.plot(t_min, xs_min[-1].detach().numpy().reshape(-1,N*2)[...,k], 'b.-', label='Final (optimized) path')
ax.plot(t_min[[0,-1]], xs_min[0].detach().numpy().reshape(-1,N*2)[...,k][[0,-1]], 'b+', markersize=15, label='Points held constant')


ax_labels = {'title':'Ball {} X-Coordinate vs. Time'.format(1 + k//2),
             'x_label':'Time (s)', 'y_label':'Position (m)'}

Plot_help(ax, ax_labels)

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=30, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
name = 'Three body'
ax_labels = {'title':'Action and associated quantities ({})'.format(name),
                 'x_label':'Optimizer Steps', 'y_label':'J * s'}
fig = action_plot(info, x_sim, ax_labels, L_fn=lagrangian_3body)
#plt.ylim(-400, 200)
path ='./static/{}_action.pdf'.format(name.lower().replace(' ', '')) ; print(path)
plt.show() #; fig.savefig(path)

## Gas simulation

In [ ]:
t, x = simulate_gas(dt=.5, N=50)

make_video(x, path='sim.mp4', interval=30)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
dt = 0.5 ; N = 50
t_sim, x_sim = simulate_gas(dt=dt, N=N)
init_path = PerturbedPath(x_sim, N=N, sigma=1e-2) # [time, N*2]
t_min, path, xs_min, info = minimize_action(init_path, steps=500, step_size=1e1,
                                       L_fn=lagrangian_gas, dt=dt, opt='sgd')

In [ ]:
N = x_sim.shape[-2]
xs_before = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_after = xs_min[-1].detach().numpy().reshape(-1,N,2)

k = 25
plt.figure(dpi=100)
plt.title('Ball {} horiz. velocity vs. time'.format(1 + k//2))
plt.plot((xs_before[1:] - xs_before[:-1]).reshape(-1,N*2)[...,k], '.-', label='Initial path')
plt.plot((xs_after[1:] - xs_after[:-1]).reshape(-1,N*2)[...,k], '.-', label='Minimum action')
plt.plot((x_sim[1:] - x_sim[:-1]).reshape(-1,N*2)[...,k], 'k-', label='Simulator')
plt.legend()
plt.show()

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=30, ms=10)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
name = 'Gas'
ax_labels = {'title':'Action and associated quantities ({})'.format(name),
                 'x_label':'Optimizer Steps', 'y_label':'J * s'}
fig = action_plot(info, x_sim, ax_labels, L_fn=lagrangian_3body)
# plt.ylim(None, 0.001)
path ='./static/{}_action.pdf'.format(name.lower().replace(' ', '')) ; print(path)
plt.show() #; fig.savefig(path)

## Ephemeris dataset and simulation

In [ ]:
planets = ['sun', 'mercury', 'venus', 'earth', 'mars']
data_dir = './data/'
df = process_raw_ephemeris(planets, data_dir, last_n_days=365) #365

t_sim, x_sim = simulate_planets(df, planets)
plot_planets(df, planets)

colors = get_planet_colors()
for i, (planet, coords) in enumerate(zip(planets, x_sim.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, ':', alpha=0.5, color=colors[planet], label=planets[i] + ' (sim)')
    plt.plot(x[0], y[0], '+', color=colors[planet])
    plt.plot(x[-1], y[-1], 'x', color=colors[planet])
plt.axis('equal')
plt.legend(fontsize=6,  loc='upper right', ncol=2) ; plt.show()

In [ ]:
dt = 24*60*60 ; N = len(planets)
df = process_raw_ephemeris(planets, data_dir, last_n_days=365)
t_sim, x_sim = simulate_planets(df, planets, dt=dt)
init_path = PerturbedPath(x_sim, N=N, sigma=2e10, is_ephemeris=True) # [time, N*2]

L_planets = partial(lagrangian_planets, masses=get_masses(planets))

t_min, path, xs_min = minimize_action(init_path, steps=500, step_size=1e9,
                                       L_fn=L_planets, dt=dt, opt='adam')

In [ ]:
plt.figure(figsize=[5,3], dpi=120)
plt.title('Earth y coordinate')
xs_sim = init_path.x_true
xs_init = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_final = xs_min[-1].detach().numpy().reshape(-1,N,2)
plt.plot(xs_sim[:,2,1], '--', label='sim')
plt.plot(xs_init[:,2,1], alpha=0.5, label='init')
plt.plot(xs_final[:,2,1], alpha=0.5, label='final')
plt.legend()

In [ ]:
fig = plt.figure(figsize=[5,5], dpi=140)
plot_planets(df, planets, fig=fig)
colors = get_planet_colors()

xs = xs_min[0].detach().numpy().reshape(-1,N,2)
for i, (planet, coords) in enumerate(zip(planets, xs.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, '.', alpha=0.3, color=colors[planet], label=planets[i] + ' (init)')
    plt.plot(x[0], y[0], '+', color=colors[planet])
    plt.plot(x[-1], y[-1], 'x', color=colors[planet])
    
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)
for i, (planet, coords) in enumerate(zip(planets, xs.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, ':', alpha=0.5, color=colors[planet], label=planets[i] + ' (path)')

plt.axis('equal')
plt.legend(fontsize=6,  loc='upper right', ncol=3) ; plt.show()